In [ ]:
#default_exp augmentation

# Tabular Augmentations
> CutMix and MixUp based on the SAINT paper

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import torch
import numpy as np

from fastcore.xtras import nested_attr
from fastai.torch_core import to_device, apply

/mnt/d/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Masking

In [ ]:
#export
def embed_data_mask(
    x_cat:torch.Tensor, # Input categorical features
    x_cont:torch.Tensor, # Input continouous features
    cat_mask:torch.Tensor, # Mask of categorical inputs
    cont_mask:torch.Tensor, # Mask of continouous inputs
    net, # The SAINT architecture
    vision_dset=False # Is the dataset a vision dataset
) -> (
    torch.Tensor, # New categorical input
    torch.Tensor, # New categorical encoding
    torch.Tensor # New continuous encoding
):
    "Embed and mask the data through the embeddings with MLP attention"
    # Can we rewrite this as a `Module`
    device = x_cont.device
    def apply_offset(model, attr, tensor_type):
        return nested_attr(f'{attr}_offset.type_as', model)(tensor_type)
    
    x_cat += apply_offset(net, 'categories', x_cat)
    x_cat_enc = net.embeds(x_cat)
    # n1,n2,n3 = BS,n_cont,n_cat
    n1,n2 = x_cont.shape
    _, n3 = x_cat.shape
    
    x_cont_enc = torch.empty(n1, n2, net.dim)
    # Do we need this for loop, can it be done quicker?
    for i in range(net.num_continuous):
        x_cont_enc[:,i,:] = net.simpleMLP[i](x_cont[:,i])
    
    x_cont_enc = x_cont_enc.to(device)
    
    cat_mask_tmp = cat_mask + apply_offset(net, 'cat_mask', cat_mask)
    con_mask_tmp = cont_mask + apply_offset(net, 'cont_mask', cont_mask)
    
    cat_mask_tmp = net.mask_embeds_cat(cat_mask_tmp)
    cont_mask_tmp = net.mask_embeds_cont(cont_mask_tmp)
    
    x_cat_enc[cat_mask == 0] = cat_mask_tmp[cat_mask == 0]
    x_cont_enc[cont_mask == 0] = cont_mask_tmp[cont_mask == 0]
    
    return x_cat, x_cat_enc, x_cont_enc

## CutMix and MixUp

In [ ]:
#export
def SAINTMixUp(
    x1: torch.Tensor, # One tensor in a batch
    x2: torch.Tensor, # Another random tensor in a batch
    lam: float = 1, # Lambda to apply on inputs
    y: torch.Tensor = None, # Targets
    use_cuda:bool = True # Whether to perform action on the GPU
) -> (
    torch.Tensor, # Mixed up `x1`
    torch.Tensor # Mixed up `x2`
):
    "Mixing up inputs, and returning a pair of targets"
    
    bs = x1.size()[0]
    device = 'cpu' if not use_cuda else 'cuda'
    idx = to_device(torch.randperm(bs), device)
    
    # Applies `mix` to both x1 and x2 on lam and at idx
    def _mix(x, lam, idx): return lam * x + (1-lam) * x[idx,:]
    mix_x1, mix_2 = apply(partial(_mix, lam=lam, idx=idx), (x1, x2))
    
    if y is not None:
        y_a, y_b = y, y[idx]
        return mix_x1, mix_x2, y_a, y_b
    
    return mix_1, mix_2

In [ ]:
#export
def SAINTCutMix(
    x_cat: torch.Tensor, # One batch of categorical data
    x_cont: torch.Tensor, # One batch of continuous data
    lam:float = 0.1 # Lambda to apply on inputs
) -> (
    torch.Tensor, # Newly noised categorical features
    torch.Tensor, # Newly noised continuous features
):
    "Applies `CutMix` on the `x's`, noisifying the data"
    bs, device = x_cat.size()[0], x_cat.device
    
    idx = torch.randperm(bs)
    def _apply_lambda(i, shape, lam, device):
        return torch.from_numpy(np.random.choice(i, shape, p=[lam,1-lam])).to(device)
    
    cat_corr = _apply_lambda(2, x_cat.shape, lam, device)
    cont_corr = _apply_lambda(2, x_cont.shape, lam, device)
    x1, x2 = x_cat[idx,:], x_cont[idx,:]
    x_cat_corr, x_cont_corr = x_cat.clone().detach(), x_cat.clone().detach()
    
    x_cat_corr[cat_corr==0] = x1[cat_corr==0]
    x_cont_cor[cont_corr==0] = x2[cont_corr==0]
    return x_cat_corr, x_cont_corr

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_augment.ipynb.
Converted index.ipynb.
Converted stream_notebook.ipynb.


In [ ]:
s